# Programmieraufgabe 10: Trust Region Verfahren

Als letztes Verfahren für unrestringierte Optimierung wollen wir nun das Trust Region Verfahren betrachten.

Tragen Sie zunächst in der folgenden Zelle **beide** Ihre Namen ein:

In [ ]:
# Numerische Optimierungsverfahren der Wirtschaftsmathematik
# Wintersemester 2025/2026
# Übungsblatt 10 - Programmieraufgabe 10
#
# [Nachname], [Vorname]
# [Vorname.Nachname@uni-a.de]
# 
# [Nachname], [Vorname]
# [Vorname.Nachname@uni-a.de]

Führen Sie die folgende Zelle zu Beginn einmal aus, selbst wenn Sie ihre Importe selbst definieren. Sie werden unter anderem auch für das Plotting benötigt.

In [ ]:
import numpy as np
import numpy.linalg as la

Beginnen Sie damit, das Trust-Region-Teilproblem (Algorithmus 3.18) zu implementieren.
Beachten Sie dabei:
- Für die Cholesky-Zerlegung können Sie `la.cholesky` verwenden, für das Lösen der linearen Gleichungssysteme `la.solve`.
- Als Abbruchkriterium in Zeile 16 verwenden Sie  $\psi(\lambda) \leq \text{tol}$.
- Als Shift $\lambda$ in Zeile 9 verwenden Sie $\max\{2|\lambda_1|, \text{lambda\_min} \}$, wobei $\lambda_1$ der kleinste Eigenwert von `H_k` ist und `lambda_min` ein Parameter der Funktion.
- Da für den Shift sowieso die Eigenwerte berechnet werden müssen, können Sie diese auch verwenden, um zu testen, ob `H_k` positiv definit ist. Als Kriterium dafür können Sie z.B. $\lambda_1 > \sqrt{\text{tol}}$ benutzen. Alternativ können Sie die Einträge der Cholesky-Zerlegung auf `nan` testen, z.B. `not np.any(np.isnan(C_k))`.

In [ ]:
def TrustRegionTeilproblem(g_k, H_k, Delta_k, tol = 1e-8, lambda_min = 1e-2):
    '''
        Lösung für das Trust-Region-Teilproblem

        Parameter:
            g_k        : Gradient als numpy-Array
            H_k        : Hesse-Matrix als 2d-numpy-Array
            Delta_k    : Größe des Trust-Region-Bereichs
            tol        : Abbruchkriterium
            lambda_min : Minimaler Shift 

        Rückgabewert:
            direction  : Lösung des Trust-Region-Teilproblems (d_k im Skript)
    '''
    
    # la.cholesky nimmt bereits implizit Symmetrie an
    if not np.array_equal(H_k, H_k.T):
        raise ValueError(f"Die Hesse-Matrix {H_k} ist nicht symmetrisch!")

    # la.eig gibt die Eigenwerte nicht unbedingt sortiert zurück
    evs = np.sort(la.eig(H_k)[0])
    assert not np.any(np.iscomplex(evs))

    ???

    return direction

Als Test bietet sich wieder eine quadratische Funktion an. Ist der Vertrauensbereich groß genug, so dass er das tatsächliche Minimum $(0, 0)$ enthält, führt die Lösung des Trust-Region-Teilproblems direkt zur Lösung. Ist er kleiner, sollte zumindest eine Verbesserung eintreten.

Beachten Sie, dass dieser Test nur als Hilfestellung gedacht ist. Selbst wenn Ihre Implementierung hier funktioniert, muss sie trotzdem noch nicht immer die korrekte Lösung liefern.

In [ ]:
f = lambda x : x[0]**2 + x[1]**2
g = lambda x : np.array([2*x[0], 2*x[1]])
h = lambda x : np.array([[2, 0], [0, 2]])

x_k = np.array([0.05, 0.1])
# Vertrauensbereich groß genug
direction_1 = TrustRegionTeilproblem(g(x_k), h(x_k), 1.0)
assert la.norm(x_k + direction_1 - np.array([0.0, 0.0])) < 1e-16

# Vertrauensbereich kleiner
direction_2 = TrustRegionTeilproblem(g(x_k), h(x_k), 0.1)
assert la.norm(x_k + direction_2 - np.array([0.0, 0.0])) > 1e-16
assert la.norm(direction_2) < 0.1

assert f(x_k) > f(x_k + direction_2)

Als zweiten Teil implementieren Sie nun das eigentliche Trust-Region-Verfahren (Algorithmus 3.17).
Beachten Sie dabei:

- In `iterates` sollen nur erfolgreiche Iterationen gezählt werden, also solche, für die die if-Bedingung in Zeile 8 wahr ist.
- Um Endlosschleifen zu vermeiden, soll die Methode nach `max_inner` nicht erfolgreichen Schritten hintereinander terminieren. Nach einem erfolgreichen Schritt wird der Zähler zurückgesetzt und es können erneut maximal `max_inner` nicht erfolgreiche Schritte erfolgen.
- Die Methode soll außerdem terminieren, wenn entweder `kmax` erfolgreiche (siehe oben) Iterationen durchgeführt worden sind oder $|f(x^{(k + 1)}) - f(x^{(k)})| < \text{tol}$ oder $|\tilde{f_k}(d^{(k)}) - f(x^{(k)})| < \text{tol}$ gilt.
- An den Stellen, an denen in Algorithmus 3.17 ein Intervall angegeben ist, wählen Sie die obere Grenze.
- Als Toleranz `tol` können Sie im Trust-Region-Teilproblem den selben Wert wie in `TrustRegion` verwenden. 

In [ ]:
def TrustRegion(
    f, g, h, x_0,
    tol = 1e-8, kmax = 50,
    delta_1 = 0.25, delta_2 = 0.75,
    sigma_1 = 0.5, sigma_2 = 2.0,
    Delta_0 = 1.0, Delta_min = 1e-3,
    lambda_min = 1e-2, max_inner = 20,
):
    '''
        Trust-Region-Verfahren

        Parameter:
            f          : Die zu minimierende Funktion f
            g          : Funktion, die den Gradienten von f zurückgibt
            h          : Funktion, die die Hesse-Matrix von f zurückgibt
            x_0        : Startvektor
            tol        : Abbruchkriterium
            kmax       : Maximale Zahl an (echten) Iterationen
            delta_1    : Parameter des Trust-Region-Verfahrens
            delta_2    : Parameter des Trust-Region-Verfahrens
            sigma_1    : Parameter des Trust-Region-Verfahrens
            sigma_1    : Parameter des Trust-Region-Verfahrens
            Delta_0    : Startgröße des Trust-Region-Bereichs
            Delta_min  : Minimaler Wert für Delta
            lambda_min : Minimaler Shift
            max_inner  : Maximale Anzahl an inneren Iterationen pro Schritt

        Rückgabewert:
            iterates : Vektor der Iterierten. Dabei ist iterates[0,:]
                       der Startwert, iterates[-1,:] der Rückgabewert
                       und len(iterates) - 1 die Anzahl an Iterationen
    '''

    iterates = np.ones((kmax + 1, x_0.shape[0]))
    x_k = np.copy(x_0)

    ???

    return iterates

Als Testfunktionen verwenden wir wieder die Rosenbrock, Himmelblau und Sattel Funktionen aus Aufgabe 7. Aus Platzgründen importieren wir sie hier, die Definition entspricht aber exakt der früheren Aufgabe.

In [ ]:
from util.functions_10 import rosenbrock_funktion, rosenbrock_gradient, rosenbrock_hesse, himmelblau_funktion, himmelblau_gradient, himmelblau_hesse, sattel_funktion, sattel_gradient, sattel_hesse

rosenbrock = rosenbrock_funktion, rosenbrock_gradient, rosenbrock_hesse
himmelblau = himmelblau_funktion, himmelblau_gradient, himmelblau_hesse
sattel = sattel_funktion, sattel_gradient, sattel_hesse

Das Trust-Region-Verfahren sollte in ca. 21 Schritten in die Nähe des Minimums (1, 1) konvergieren:

In [ ]:
iterates = TrustRegion(*rosenbrock, np.array([-1.3, 1.0]))
print(f'Das Trust-Region-Verfahren konvergiert nach {len(iterates) - 1} Schritten gegen {iterates[-1]}.')

Abschließend wollen wir wie in den früheren Aufgaben die Iterationszahlen der Verfahren auf den Testfunktionen darstellen.

In [ ]:
# Diese Zeilen werden für die plots benötigt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from util.plotting_10 import plot_10

resolution = (45, 30)

In [ ]:
x_range = (-1.5, 1.5)
y_range = (-0.4, 1.6)
plot_10(TrustRegion, *rosenbrock, *resolution, *x_range, *y_range)

In [ ]:
x_range = (-7.5, 7.5)
y_range = (-5.0, 5.0)
plot_10(TrustRegion, *himmelblau, *resolution, *x_range, *y_range)

In [ ]:
x_range = (-9.0, 6.0)
y_range = (-5.0, 5.0)
plot_10(TrustRegion, *sattel, *resolution, *x_range, *y_range)